In [1]:
import numpy as np
import pandas as pd
import os
import pickle
from collections import OrderedDict

In [2]:
# note to self: 'Lateral Transport, Advection' and 'Lateral Transport, Mixing' are of NO3 only

In [3]:
sums=dict()
sums[2015]=pickle.load( open( "sums2015.pkl", "rb" ) )
sums[2016]=pickle.load( open( "sums2016.pkl", "rb" ) )
sums[2017]=pickle.load( open( "sums2017.pkl", "rb" ) )

In [4]:
print(sums[2015].keys())
for el in sums[2015].keys():
    print(el)

dict_keys(['Lateral Transport, Advection', 'Lateral Transport, Mixing', 'Rivers NO3', 'NO3-based Primary Production', 'NH4-based Primary Production', 'Nitrification', 'Net change', 'DIN (NH4+NO3) from Rivers', 'Haro Strait NO3', 'Haro Strait NH4', 'Haro Strait DIN', 'Haro Strait PON', 'Haro Strait DON', 'Haro Strait Living', 'Southern NO3', 'Southern NH4', 'Southern DIN', 'Southern PON', 'Southern DON', 'Southern Living', 'Northern NO3', 'Northern NH4', 'Northern DIN', 'Northern PON', 'Northern DON', 'Northern Living', 'PON&DON->NH4', 'To Refractory N', 'Bottom flux diatoms', 'Bottom flux PON', 'Deep Haro Strait NO3', 'Deep Haro Strait NH4', 'Deep Haro Strait DIN', 'Deep Haro Strait PON', 'Deep Haro Strait DON', 'Deep Haro Strait Living', 'Net change NO3'])
Lateral Transport, Advection
Lateral Transport, Mixing
Rivers NO3
NO3-based Primary Production
NH4-based Primary Production
Nitrification
Net change
DIN (NH4+NO3) from Rivers
Haro Strait NO3
Haro Strait NH4
Haro Strait DIN
Haro Stra

In [5]:
len(' '.strip())

0

In [6]:
def kmps(Gmolperday):
    # non leap years
    #convert Gmol/year to kmol/s
    # Gmol/yr* 1e6 kmol/Gmol * 1/(3600*24*365 s/yr)
    kmolpers=Gmolperday*1e6/(3600*24*365)
    return kmolpers
def kmps16(Gmolperday):
    # 2016 was leap year
    #convert Gmol/year to kmol/s
    # Gmol/yr* 1e6 kmol/Gmol * 1/(3600*24*366 s/yr)
    kmolpers=Gmolperday*1e6/(3600*24*366)
    return kmolpers
def nmperm2s(Gmolperday):
    # non leap years
    #convert Gmol/year to nmol/m2/s
    # Gmol/yr* 1e6 kmol/Gmol * 1/(3600*24*365 s/yr)*1e12nmol/kmol/( m^2)
    kmolpers=Gmolperday*1e6/(3600*24*365)*1e12/(9815*1e6)
    return kmolpers
def nmperm2s16(Gmolperday):
    # 2016 was leap year
    #convert Gmol/year to nmol/m2/s
    # Gmol/yr* 1e6 kmol/Gmol * 1/(3600*24*366 s/yr)*1e12nmol/kmol/( m^2)
    kmolpers=Gmolperday*1e6/(3600*24*366)*1e12/(9815*1e6)
    return kmolpers

In [7]:
class tblList:
    # Initializer / Instance Attributes
    def __init__(self, headings, fname):
        self.lines = list()
        self.lines.append(r'   \small'+'\n')
        self.lines.append(r'   \centering'+'\n')
        self.lines.append(r'   \begin{tabular}{lrcccccccccccc}\toprule'+'\n')
        self.lines.append((r'      &  &  & \multicolumn{3}{c}{ Gmol N yr$^{-1}$ }'
                          r' &  & \multicolumn{3}{c}{ kmol N s$^{-1}$ } '
                          r' &  & \multicolumn{3}{c}{ nmol N m$^{-2}$s$^{-1}$ } \\'+'\n'))
        if len(headings[0].strip())>0:
            self.lines.append(f'    \multicolumn{{2}}{{l}}{{ {headings[0]} }}  & \phantom{{a}} ' 
                              f' & {headings[1]} & {headings[2]} & {headings[3]}& \phantom{{a}} '
                              f' & {headings[1]} & {headings[2]} & {headings[3]}& \phantom{{a}} '
                              f' & {headings[1]} & {headings[2]} & {headings[3]} \\\\ \midrule'+'\n')
        else:
            self.lines.append(f'   &   & \phantom{a}  & {headings[1]} & {headings[2]} & {headings[3]} & \phantom{a} '
                              f' & {headings[1]} & {headings[2]} & {headings[3]}& \phantom{{a}} '
                              f' & {headings[1]} & {headings[2]} & {headings[3]} \\\\ \midrule'+'\n')
        self.fname = fname
        
    def addlines(self,heading,val2015,val2016,val2017):
        if val2015>10:
            self.lines.append(f'     \multicolumn{{2}}{{l}}{{ {heading} }} '
                              f'& & {val2015:.1f} & {val2016:.1f} & {val2017:.1f}  '
                              f'& & {kmps(val2015):.2f} & {kmps16(val2016):.2f} & {kmps(val2017):.2f} '
                              f'& &  &  &  \\\\'+'\n')
        else:
            self.lines.append(f'     \multicolumn{{2}}{{l}}{{ {heading} }} '
                              f'& & {val2015:.2f} & {val2016:.2f} & {val2017:.2f} '
                              f'& & {kmps(val2015):.3f} & {kmps16(val2016):.3f} & {kmps(val2017):.3f} '
                              f'& &  &  &  \\\\'+'\n')

    def addlinesSub(self,heading,val2015,val2016,val2017):
        if val2015>10:
            self.lines.append(f'       & {heading} '
                              f'& & {val2015:.1f} & {val2016:.1f} & {val2017:.1f} '
                              f'& & {kmps(val2015):.2f} & {kmps16(val2016):.2f} & {kmps(val2017):.2f} '
                              f'& &  &  &  \\\\'+'\n')
        else:
            self.lines.append(f'       & {heading} '
                              f'& & {val2015:.2f} & {val2016:.2f} & {val2017:.2f} '
                              f'& & {kmps(val2015):.3f} & {kmps16(val2016):.3f} & {kmps(val2017):.3f} '
                              f'& &  &  &  \\\\'+'\n')
                
    def addlinesAreal(self,heading,val2015,val2016,val2017):
        if val2015>10:
            self.lines.append(f'     \multicolumn{{2}}{{l}}{{ {heading} }} '
                              f'& & {val2015:.1f} & {val2016:.1f} & {val2017:.1f}  '
                              f'& & {kmps(val2015):.2f} & {kmps16(val2016):.2f} & {kmps(val2017):.2f} '
                              f'& & {nmperm2s(val2015):.3g} & {nmperm2s16(val2016):.3g} & {nmperm2s(val2017):.3g} \\\\'+'\n')
        else:
            self.lines.append(f'     \multicolumn{{2}}{{l}}{{ {heading} }} '
                              f'& & {val2015:.2f} & {val2016:.2f} & {val2017:.2f} '
                              f'& & {kmps(val2015):.3f} & {kmps16(val2016):.3f} & {kmps(val2017):.3f} '
                              f'& & {nmperm2s(val2015):.3g} & {nmperm2s16(val2016):.3g} & {nmperm2s(val2017):.3g} \\\\'+'\n')

    def addlinesSubAreal(self,heading,val2015,val2016,val2017):
        if val2015>10:
            self.lines.append(f'       & {heading} '
                              f'& & {val2015:.1f} & {val2016:.1f} & {val2017:.1f} '
                              f'& & {kmps(val2015):.2f} & {kmps16(val2016):.2f} & {kmps(val2017):.2f} '
                              f'& & {nmperm2s(val2015):.3g} & {nmperm2s16(val2016):.3g} & {nmperm2s(val2017):.3g} \\\\'+'\n')
        else:
            self.lines.append(f'       & {heading} '
                              f'& & {val2015:.2f} & {val2016:.2f} & {val2017:.2f} '
                              f'& & {kmps(val2015):.3f} & {kmps16(val2016):.3f} & {kmps(val2017):.3f} '
                              f'& & {nmperm2s(val2015):.3g} & {nmperm2s16(val2016):.3g} & {nmperm2s(val2017):.3g} \\\\'+'\n')
        
    def finish(self):
        self.lines.append(r'   \midrule'+'\n')
        self.lines.append(r' \\ \\')
        self.lines.append(r'   \end{tabular}'+'\n')
        
    def write(self):
        f = open(self.fname, 'w')
        f.writelines(self.lines)
        f.close()

In [8]:
net2015=sums[2015]['Bottom flux diatoms']+sums[2015]['Bottom flux PON']+sums[2015]['DIN (NH4+NO3) from Rivers']+\
         sums[2015]['Southern NO3']+sums[2015]['Southern NH4']+\
        sums[2015]['Southern PON']+sums[2015]['Southern DON']+sums[2015]['Southern Living']+\
         sums[2015]['Northern NO3']+sums[2015]['Northern NH4']+\
        sums[2015]['Northern PON']+sums[2015]['Northern DON']+sums[2015]['Northern Living']
net2016=sums[2016]['Bottom flux diatoms']+sums[2016]['Bottom flux PON']+sums[2016]['DIN (NH4+NO3) from Rivers']+\
         sums[2016]['Southern NO3']+sums[2016]['Southern NH4']+\
        sums[2016]['Southern PON']+sums[2016]['Southern DON']+sums[2016]['Southern Living']+\
         sums[2016]['Northern NO3']+sums[2016]['Northern NH4']+\
        sums[2016]['Northern PON']+sums[2016]['Northern DON']+sums[2016]['Northern Living']
net2017=sums[2017]['Bottom flux diatoms']+sums[2017]['Bottom flux PON']+sums[2017]['DIN (NH4+NO3) from Rivers']+\
         sums[2017]['Southern NO3']+sums[2017]['Southern NH4']+\
        sums[2017]['Southern PON']+sums[2017]['Southern DON']+sums[2017]['Southern Living']+\
         sums[2017]['Northern NO3']+sums[2017]['Northern NH4']+\
        sums[2017]['Northern PON']+sums[2017]['Northern DON']+sums[2017]['Northern Living']

In [9]:
# transfer NO3 sums from other notebooks:
NO3ContentDiff2015=-1117917.27366437*24*3600*1e-12
NO3ContentDiff2016=9725303.241466291*24*3600*1e-12
NO3ContentDiff2017=8487324.512537435*24*3600*1e-12

In [10]:
NO3ContentDiff2015

-0.09658805244460156

In [11]:
netNO32015=sums[2015]['NO3-based Primary Production']+sums[2015]['Nitrification']+\
            sums[2015]['Rivers NO3']+sums[2015]['Southern NO3']+sums[2015]['Northern NO3']
netNO32016=sums[2016]['NO3-based Primary Production']+sums[2016]['Nitrification']+\
            sums[2016]['Rivers NO3']+sums[2016]['Southern NO3']+sums[2016]['Northern NO3']
netNO32017=sums[2017]['NO3-based Primary Production']+sums[2017]['Nitrification']+\
            sums[2017]['Rivers NO3']+sums[2017]['Southern NO3']+sums[2017]['Northern NO3']

In [12]:
netNO32015

-0.013938507647996312

In [13]:
tbl=tblList(('Budget Component','2015','2016','2017'),'fluxesTBL.tex')
tbl.addlinesAreal('Primary Production',
                -1*(sums[2015]['NO3-based Primary Production']+sums[2015]['NH4-based Primary Production']),
                -1*(sums[2016]['NO3-based Primary Production']+sums[2016]['NH4-based Primary Production']),
                -1*(sums[2017]['NO3-based Primary Production']+sums[2017]['NH4-based Primary Production']))
tbl.addlinesSubAreal('      NO$_3$-based',
                -1*sums[2015]['NO3-based Primary Production'],
                -1*sums[2016]['NO3-based Primary Production'],
                -1*sums[2017]['NO3-based Primary Production'])
tbl.addlinesSubAreal('      NH$_4$-based',
                -1*sums[2015]['NH4-based Primary Production'],
                -1*sums[2016]['NH4-based Primary Production'],
                -1*sums[2017]['NH4-based Primary Production'])
tbl.addlinesAreal('Remineralization (to NH$_4$)',
                -1*sums[2015]['PON&DON->NH4'],
                -1*sums[2016]['PON&DON->NH4'],
                -1*sums[2017]['PON&DON->NH4'])
tbl.addlinesAreal('Nitrification',
                sums[2015]['Nitrification'],
                sums[2016]['Nitrification'],
                sums[2017]['Nitrification'])
#tbl.addlines('Remineralization (to NH$_4$)',
#                sums[2015]['PON&DON->NH4'],
#                sums[2016]['PON&DON->NH4'],
#                sums[2017]['PON&DON->NH4'])
#tbl.addlines('Loss to Refractory Pool',
#                sums[2015]['To Refractory N'],
#                sums[2016]['To Refractory N'],
#                sums[2017]['To Refractory N'])
tbl.addlines('Loss to Sediments',
                -1*(sums[2015]['Bottom flux diatoms']+sums[2015]['Bottom flux PON']),
                -1*(sums[2016]['Bottom flux diatoms']+sums[2016]['Bottom flux PON']),
                -1*(sums[2017]['Bottom flux diatoms']+sums[2017]['Bottom flux PON']))
tbl.addlinesSub('      diatoms',
                -1*sums[2015]['Bottom flux diatoms'],
                -1*sums[2016]['Bottom flux diatoms'],
                -1*sums[2017]['Bottom flux diatoms'])
tbl.addlinesSub('      detrital PON',
                -1*sums[2015]['Bottom flux PON'],
                -1*sums[2016]['Bottom flux PON'],
                -1*sums[2017]['Bottom flux PON'])
tbl.addlines('Rivers (DIN)',
                sums[2015]['DIN (NH4+NO3) from Rivers'],
                sums[2016]['DIN (NH4+NO3) from Rivers'],
                sums[2017]['DIN (NH4+NO3) from Rivers'])
tbl.addlinesSub('NO$_3$',
                sums[2015]['Rivers NO3'],
                sums[2016]['Rivers NO3'],
                sums[2017]['Rivers NO3'])
tbl.addlinesSub('NH$_4$',
                sums[2015]['DIN (NH4+NO3) from Rivers']-sums[2015]['Rivers NO3'],
                sums[2016]['DIN (NH4+NO3) from Rivers']-sums[2016]['Rivers NO3'],
                sums[2017]['DIN (NH4+NO3) from Rivers']-sums[2017]['Rivers NO3'])
tbl.addlines('Deep Transport, Haro Strait',
        sums[2015]['Deep Haro Strait NO3']+sums[2015]['Deep Haro Strait NH4']+\
        sums[2015]['Deep Haro Strait PON']+sums[2015]['Deep Haro Strait DON']+sums[2015]['Deep Haro Strait Living'],
        sums[2016]['Deep Haro Strait NO3']+sums[2016]['Deep Haro Strait NH4']+\
        sums[2016]['Deep Haro Strait PON']+sums[2016]['Deep Haro Strait DON']+sums[2016]['Deep Haro Strait Living'],
        sums[2017]['Deep Haro Strait NO3']+sums[2017]['Deep Haro Strait NH4']+\
        sums[2017]['Deep Haro Strait PON']+sums[2017]['Deep Haro Strait DON']+sums[2017]['Deep Haro Strait Living'])
tbl.addlinesSub('NO$_3$',
        sums[2015]['Deep Haro Strait NO3'],
        sums[2016]['Deep Haro Strait NO3'],
        sums[2017]['Deep Haro Strait NO3'])
tbl.addlinesSub('NH$_4$',
        sums[2015]['Deep Haro Strait NH4'],
        sums[2016]['Deep Haro Strait NH4'],
        sums[2017]['Deep Haro Strait NH4'])
tbl.addlinesSub('detrital PON',
        sums[2015]['Deep Haro Strait PON'],
        sums[2016]['Deep Haro Strait PON'],
        sums[2017]['Deep Haro Strait PON'])
tbl.addlinesSub('DON',
        sums[2015]['Deep Haro Strait DON'],
        sums[2016]['Deep Haro Strait DON'],
        sums[2017]['Deep Haro Strait DON'])
tbl.addlinesSub('Living',
        sums[2015]['Deep Haro Strait Living'],
        sums[2016]['Deep Haro Strait Living'],
        sums[2017]['Deep Haro Strait Living'])
tbl.addlines('Total Transport, Haro Strait',
        sums[2015]['Haro Strait NO3']+sums[2015]['Haro Strait NH4']+\
        sums[2015]['Haro Strait PON']+sums[2015]['Haro Strait DON']+sums[2015]['Haro Strait Living'],
        sums[2016]['Haro Strait NO3']+sums[2016]['Haro Strait NH4']+\
        sums[2016]['Haro Strait PON']+sums[2016]['Haro Strait DON']+sums[2016]['Haro Strait Living'],
        sums[2017]['Haro Strait NO3']+sums[2017]['Haro Strait NH4']+\
        sums[2017]['Haro Strait PON']+sums[2017]['Haro Strait DON']+sums[2017]['Haro Strait Living'])
tbl.addlinesSub('NO$_3$',
        sums[2015]['Haro Strait NO3'],
        sums[2016]['Haro Strait NO3'],
        sums[2017]['Haro Strait NO3'])
tbl.addlinesSub('NH$_4$',
        sums[2015]['Haro Strait NH4'],
        sums[2016]['Haro Strait NH4'],
        sums[2017]['Haro Strait NH4'])
tbl.addlinesSub('detrital PON',
        sums[2015]['Haro Strait PON'],
        sums[2016]['Haro Strait PON'],
        sums[2017]['Haro Strait PON'])
tbl.addlinesSub('DON',
        sums[2015]['Haro Strait DON'],
        sums[2016]['Haro Strait DON'],
        sums[2017]['Haro Strait DON'])
tbl.addlinesSub('Living',
        sums[2015]['Haro Strait Living'],
        sums[2016]['Haro Strait Living'],
        sums[2017]['Haro Strait Living'])
tbl.addlines('Total Transport, Southern Straits',
        sums[2015]['Southern NO3']+sums[2015]['Southern NH4']+\
        sums[2015]['Southern PON']+sums[2015]['Southern DON']+sums[2015]['Southern Living'],
        sums[2016]['Southern NO3']+sums[2016]['Southern NH4']+\
        sums[2016]['Southern PON']+sums[2016]['Southern DON']+sums[2016]['Southern Living'],
        sums[2017]['Southern NO3']+sums[2017]['Southern NH4']+\
        sums[2017]['Southern PON']+sums[2017]['Southern DON']+sums[2017]['Southern Living'])
tbl.addlinesSub('NO$_3$',
        sums[2015]['Southern NO3'],
        sums[2016]['Southern NO3'],
        sums[2017]['Southern NO3'])
tbl.addlinesSub('NH$_4$',
        sums[2015]['Southern NH4'],
        sums[2016]['Southern NH4'],
        sums[2017]['Southern NH4'])
tbl.addlinesSub('detrital PON',
        sums[2015]['Southern PON'],
        sums[2016]['Southern PON'],
        sums[2017]['Southern PON'])
tbl.addlinesSub('DON',
        sums[2015]['Southern DON'],
        sums[2016]['Southern DON'],
        sums[2017]['Southern DON'])
tbl.addlinesSub('Living',
        sums[2015]['Southern Living'],
        sums[2016]['Southern Living'],
        sums[2017]['Southern Living'])
tbl.addlines('Total Transport, Northern Straits',
        sums[2015]['Northern NO3']+sums[2015]['Northern NH4']+\
        sums[2015]['Northern PON']+sums[2015]['Northern DON']+sums[2015]['Northern Living'],
        sums[2016]['Northern NO3']+sums[2016]['Northern NH4']+\
        sums[2016]['Northern PON']+sums[2016]['Northern DON']+sums[2016]['Northern Living'],
        sums[2017]['Northern NO3']+sums[2017]['Northern NH4']+\
        sums[2017]['Northern PON']+sums[2017]['Northern DON']+sums[2017]['Northern Living'])
tbl.addlinesSub('NO$_3$',
        sums[2015]['Northern NO3'],
        sums[2016]['Northern NO3'],
        sums[2017]['Northern NO3'])
tbl.addlinesSub('NH$_4$',
        sums[2015]['Northern NH4'],
        sums[2016]['Northern NH4'],
        sums[2017]['Northern NH4'])
tbl.addlinesSub('detrital PON',
        sums[2015]['Northern PON'],
        sums[2016]['Northern PON'],
        sums[2017]['Northern PON'])
tbl.addlinesSub('DON',
        sums[2015]['Northern DON'],
        sums[2016]['Northern DON'],
        sums[2017]['Northern DON'])
tbl.addlinesSub('Living',
        sums[2015]['Northern Living'],
        sums[2016]['Northern Living'],
        sums[2017]['Northern Living'])
#tbl.addlines('Net Change',
#                sums[2015]['Net change'],
#                sums[2016]['Net change'],
#                sums[2017]['Net change'])
tbl.addlines('Loss to Refractory N',
                -1*(sums[2015]['Net change']-net2015),
                -1*(sums[2016]['Net change']-net2016),
                -1*(sums[2017]['Net change']-net2017))
tbl.addlines('Residual NO$_3$ Change',
                NO3ContentDiff2015-netNO32015,
                NO3ContentDiff2016-netNO32016,
                NO3ContentDiff2017-netNO32017)
tbl.finish()
tbl.write()

In [14]:
net2015

0.4001005018452666

In [15]:
-6.54+1.64+2.75+2.55

0.39999999999999947

In [16]:
f'{159.8845:.3g}'

'160'